In [3]:
import pandas as pd

In [4]:
excel_file_path = 'C:/Users/ASUS/Documents/實習/Rebas/CPBL_2023.xlsx'  # 請替換成你的 Excel 檔案路徑
sheet_name1 ='AWAY-打席資訊'  # 請替換成你要載入的工作表名稱
sheet_name2 ='HOME-打席資訊'

In [5]:
away = pd.read_excel(excel_file_path, sheet_name=sheet_name1)
home = pd.read_excel(excel_file_path, sheet_name=sheet_name2)

In [6]:
cols = ['客場','主場','開始出局數','結束出局數','打席結果'] 

In [7]:
away_play = away[cols]
home_play = home[cols]

In [8]:
away_play = away_play.rename(columns={'客場': '球隊', '主場': '對手'})
home_play = home_play.rename(columns={'主場': '球隊', '客場': '對手'})

In [9]:
away_play

,球隊,對手,開始出局數,結束出局數,打席結果
0,富邦悍將,統一7-ELEVEn獅,0,1,FO
1,富邦悍將,統一7-ELEVEn獅,1,1,1B
2,富邦悍將,統一7-ELEVEn獅,1,1,1B
3,富邦悍將,統一7-ELEVEn獅,1,2,FO
4,富邦悍將,統一7-ELEVEn獅,2,3,GO
...,...,...,...,...,...
11752,樂天桃猿,中信兄弟,1,2,FO
11753,樂天桃猿,中信兄弟,2,3,FO
11754,樂天桃猿,中信兄弟,0,1,FO
11755,樂天桃猿,中信兄弟,1,2,GO


In [10]:
ball_in_play = pd.concat([home_play,away_play], ignore_index=False)

In [9]:
ball_in_play['出局數']=ball_in_play['結束出局數']-ball_in_play['開始出局數']
ball_in_play = ball_in_play[ball_in_play["打席結果"] != "SO"]
ball_in_play = ball_in_play[ball_in_play["打席結果"] != "uBB"]
ball_in_play = ball_in_play[ball_in_play["打席結果"] != "HBP"]
ball_in_play = ball_in_play[ball_in_play["打席結果"] != "IBB"]
ball_in_play = ball_in_play[ball_in_play["打席結果"] != "HR"]


In [10]:
ball_in_play

,對手,球隊,開始出局數,結束出局數,打席結果,出局數
0,富邦悍將,統一7-ELEVEn獅,0,1,FO,1
1,富邦悍將,統一7-ELEVEn獅,1,2,FO,1
2,富邦悍將,統一7-ELEVEn獅,2,3,GO,1
4,富邦悍將,統一7-ELEVEn獅,0,1,FO,1
6,富邦悍將,統一7-ELEVEn獅,2,3,GO,1
...,...,...,...,...,...,...
11751,中信兄弟,樂天桃猿,0,1,FO,1
11752,中信兄弟,樂天桃猿,1,2,FO,1
11753,中信兄弟,樂天桃猿,2,3,FO,1
11754,中信兄弟,樂天桃猿,0,1,FO,1


In [11]:
ball_in_play['打席結果'].value_counts()

FO       5580
GO       4163
1B       4002
SO       3866
uBB      1820
2B        845
FC        699
GIDP      479
HR        329
HBP       321
SH        292
E         238
SF        184
3B        102
IBB        39
SH_FC      14
SH_E       13
ID          5
TP          1
SF_E        1
Name: 打席結果, dtype: int64

In [12]:
dfs = {}

# 定義你想分組的欄位
group_columns = ['對手']

# 分組和聚合計算出局數的次數和總和
for name, group in ball_in_play.groupby(group_columns):
    df1 = group.groupby(group_columns).agg({'出局數':'count'})
    df2 = group.groupby(group_columns).agg({'出局數':'sum'})
    dfs[name] = pd.merge(df1, df2, on=group_columns)

# 合併所有的 DataFrame
result = pd.concat(dfs.values())

result

C:\Users\ASUS\AppData\Local\Temp\ipykernel_4068\3740936095.py:7: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for name, group in ball_in_play.groupby(group_columns):


KeyError: "Column(s) ['出局數'] do not exist"

In [13]:
result = result.rename(columns={'出局數_x': 'ball_in_play', '出局數_y': '出局數'})
result

,ball_in_play,出局數
對手,,
中信兄弟,3269,2371
味全龍,3278,2401
富邦悍將,3366,2426
樂天桃猿,3541,2491
統一7-ELEVEn獅,3242,2420


In [14]:
result['DER'] = result['出局數']/result['ball_in_play']

In [17]:
result

,ball_in_play,出局數,DER
對手,,,
中信兄弟,3269,2371,0.725298
味全龍,3278,2401,0.732459
富邦悍將,3366,2426,0.720737
樂天桃猿,3541,2491,0.703474
統一7-ELEVEn獅,3242,2420,0.746453


In [19]:
result = result.sort_values(by='DER', ascending=False)
result

,ball_in_play,出局數,DER
對手,,,
統一7-ELEVEn獅,3242,2420,0.746453
味全龍,3278,2401,0.732459
中信兄弟,3269,2371,0.725298
富邦悍將,3366,2426,0.720737
樂天桃猿,3541,2491,0.703474


In [16]:
def calculate_so_go_count(series):
    so_count = (series == 'SO').sum()
    go_count = (series == 'GO').sum()
    gidp_count = (series == 'GIDP').sum()
    fo_count = (series == 'FO').sum()
    sh_count = (series == 'SH').sum()
    sf_count = (series == 'SF').sum()
    hit_count = (series == '1B').sum()
    hit2_count = (series == '2B').sum()
    hit3_count = (series == '3B').sum()
    hit4_count = (series == 'HR').sum()
    ubb_count = (series == 'uBB').sum()
    ibb_count = (series == 'IBB').sum()
    e_count = (series == 'E').sum()
    fc_count = (series == 'FC').sum()
    hbp_count = (series == 'HBP').sum()
    she_count = (series == 'SH_E').sum()
    shfc_count = (series == 'SH_FC').sum()
    return pd.Series({'SO': so_count, 'GO': go_count ,'GIDP':gidp_count, 'FO':fo_count, 'SH':sh_count, 'SF':sf_count, '1B':hit_count
                     , '2B':hit2_count
                     , '3B':hit3_count
                     , 'HR':hit4_count
                     , 'uBB':ubb_count
                    ,'IBB':ibb_count
                     , 'E':e_count
                     , 'FC':fc_count
                     , 'HBP':hbp_count
                     , 'SH_E':she_count
                     , 'SH_FC':shfc_count})

In [17]:
resault=ball_in_play.groupby(['對手'])['打席結果'].apply(calculate_so_go_count).reset_index()
resault = resault.rename(columns={'level_1': '打席結果', '打席結果': '次數'})
resault

,對手,打席結果,次數
0,中信兄弟,SO,852
1,中信兄弟,GO,808
2,中信兄弟,GIDP,106
3,中信兄弟,FO,1075
4,中信兄弟,SH,63
...,...,...,...
80,統一7-ELEVEn獅,E,42
81,統一7-ELEVEn獅,FC,103
82,統一7-ELEVEn獅,HBP,53
83,統一7-ELEVEn獅,SH_E,4


In [18]:
resault['次數'] = resault['次數'].astype(int)

final_resault = pd.pivot_table(resault, index=['對手'], columns='打席結果', values='次數', aggfunc='sum', fill_value=0)

final_resault

打席結果,1B,2B,3B,E,FC,FO,GIDP,GO,HBP,HR,IBB,SF,SH,SH_E,SH_FC,SO,uBB
對手,,,,,,,,,,,,,,,,,
中信兄弟,793,185,18,26,135,1075,106,808,61,75,11,41,63,1,3,852,334
味全龍,745,194,23,48,120,1128,97,818,71,58,8,35,53,2,3,770,361
富邦悍將,821,162,21,49,157,1034,85,919,67,52,11,36,56,3,0,738,376
樂天桃猿,938,141,18,73,184,1114,100,866,69,71,5,31,55,3,3,749,381
統一7-ELEVEn獅,705,163,22,42,103,1229,91,752,53,73,4,41,65,4,5,757,368


In [23]:
final_resault['AB']=final_resault['1B']+final_resault['2B']+final_resault['3B']+final_resault['E']+final_resault['FC']+final_resault['FO']+final_resault['GIDP']+final_resault['GO']+final_resault['HR']+final_resault['SO']
final_resault['PA']=final_resault['AB']+final_resault['HBP']+final_resault['SF']+final_resault['SH']+final_resault['SH_E']+final_resault['SH_FC']+final_resault['uBB']
final_resault['DER']=1-((final_resault['1B']+final_resault['2B']+final_resault['3B']+final_resault['E']+final_resault['SH_E'])/(final_resault['PA']-final_resault['uBB']-final_resault['IBB']-final_resault['SO']-final_resault['HBP']-final_resault['HR']))

In [22]:
final_resault = final_resault.sort_values(by='DER', ascending=False)
final_resault

打席結果,1B,2B,3B,E,FC,FO,GIDP,GO,HBP,HR,IBB,SF,SH,SH_E,SH_FC,SO,uBB,AB,PA,DER
對手,,,,,,,,,,,,,,,,,,,,
統一7-ELEVEn獅,705,163,22,42,103,1229,91,752,53,73,4,41,65,4,5,757,368,3937,4473,0.709136
味全龍,745,194,23,48,120,1128,97,818,71,58,8,35,53,2,3,770,361,4001,4526,0.689380
中信兄弟,793,185,18,26,135,1075,106,808,61,75,11,41,63,1,3,852,334,4073,4576,0.684551
富邦悍將,821,162,21,49,157,1034,85,919,67,52,11,36,56,3,0,738,376,4038,4576,0.683073
樂天桃猿,938,141,18,73,184,1114,100,866,69,71,5,31,55,3,3,749,381,4254,4796,0.666856
